In [1]:
# import packages
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub

from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.models import Sequential
from keras.metrics import sparse_top_k_categorical_accuracy
from sklearn.preprocessing import normalize
from tensorflow.python.saved_model import loader_impl
from tensorflow.python.keras.saving.saved_model import load as saved_model_load

In [2]:
#load models

cwd = os.getcwd()+'/final/'
ResNet = tf.keras.models.load_model(cwd+'ResNet.h5')
MobileNet = tf.keras.models.load_model(cwd+'MobileNet.h5')
NASNet = tf.keras.models.load_model(cwd+'NASNet.h5')

models = [ResNet, MobileNet, NASNet]

In [6]:
ResNet.summary()
MobileNet.summary()
NASNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
random_zoom (RandomZoom)     (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 80)                102480    
Total params: 2,360,464
Trainable params: 102,480
Non-trainable params: 2,257,984
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)      

In [3]:
# lood data directory to obtain labels
train_path = os.getcwd() + '/data/train_set/'+'train/'

img_height = 224
img_width = 224
IMAGE_SHAPE = (img_height, img_width)

batch_size = 64
seed = 16

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    labels='inferred', 
    label_mode='int',
    color_mode='rgb',
)

class_names = train_ds.class_names
nr_labels = len(class_names)
print('lables:', class_names)

Found 30612 files belonging to 80 classes.
Using 24490 files for training.
lables: ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '9']


In [4]:
# combine the classifiers to compute the best average calculation

def predict(file, models, test):
    img = tf.keras.preprocessing.image.load_img(file, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) 
        
        
    predictions_1 = models[0].predict(img_array)
    predictions_2 = models[1].predict(img_array)
    predictions_3 = models[2].predict(img_array)
    
    
    
    # nomralize
    norm_1 = predictions_1[0] / np.linalg.norm(predictions_1[0])
    norm_2 = predictions_2[0] / np.linalg.norm(predictions_2[0])
    norm_3 = predictions_3[0] / np.linalg.norm(predictions_3[0])
    
    predictions_4 = norm_1 + norm_2  +  norm_3
    
    max_1 = np.argmax(norm_1)
    max_2 = np.argmax(norm_2)
    max_3 = np.argmax(norm_3)
    
    if test == True:
        print('Resnet: label:', class_names[max_1],'          ,with value:' ,norm_1[max_1])
        print('MobileNet: label:',class_names[max_2], '       ,with value: ' ,norm_2[max_2])
        print('Nasnet: label:',class_names[max_3], '          ,with value: ' ,norm_3[max_3])
        print('Combined: label:',class_names[np.argmax(predictions_4)])
        
    return class_names[np.argmax(predictions_4)]
    

In [5]:
test_image = os.getcwd()+'/data/test_set/test_set/test_9.jpg'

predict(test_image, models, True)

Resnet: label: 3           ,with value: 0.28782058
MobileNet: label: 5        ,with value:  0.21792836
Nasnet: label: 3           ,with value:  0.21211657
Combined: label: 3


'3'

In [6]:
import csv
#create submission and predict for eacht test image

rootdir =os.getcwd()+'/data/test_set/test_set'

header = ["img_name", "label"]
with open('submission_ResNet_MobileNet_AC.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')
    
    filewriter.writerow(header)
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            label = predict(rootdir+'/'+file, models, False)
            filewriter.writerow([file, label])
    